# Storing Teletry


Recover data from the previous steps, make sure you've been through them and understand what they are:

In [1]:
#Hour and minute when this started to create unique names
%store -r HM
#Home directory of the user running this notebook
%store -r home
#Private key file name, for AWS IoT authentication
%store -r private_key
#Certificate file name, for AWS IoT authentication
%store -r certificate_file
#Certificate authority file name, for AWS IoT authentication
%store -r ca_file
#Address to connect using MQTT
%store -r endpoint_address
#The name of the topic to send MQTT messages to
%store -r topic_name
#A list of sample messages
%store -r payloads
#A sample message payload
%store -r payload

Create a DynamoDB table to store our messages:

In [2]:
table_name = f"bloodpressure{HM}"
table_name

'bloodpressure0524'

In [3]:
!echo Creating table "{table_name}" this takes a few seconds...
table_arn = !aws dynamodb create-table \
  --table-name {table_name} \
  --attribute-definitions \
    AttributeName=name,AttributeType=S \
    AttributeName=time_stamp,AttributeType=N \
  --key-schema AttributeName=name,KeyType=HASH AttributeName=time_stamp,KeyType=RANGE \
  --provisioned-throughput ReadCapacityUnits=1,WriteCapacityUnits=1 \
  --query TableDescription.TableArn \
  --output text
table_arn = table_arn.s
!aws dynamodb wait table-exists --table-name {table_name}
!echo Table created "{table_arn}"

Creating table bloodpressure0524 this takes a few seconds...
Table created arn:aws:dynamodb:us-east-1:030555009967:table/bloodpressure0524


Create a rule capturing messages from the topic. The rule conditions are expressed in SQL statements like the one below. Note that they can not only capture but also enrich data, see the [AWS IoT SQL Reference
](https://docs.aws.amazon.com/iot/latest/developerguide/iot-sql-reference.html) for more information.

In [4]:
sql = f"""
SELECT name, 
    time_stamp, 
    sistolic, 
    diastolic, 
    id, 
    timestamp() as recv_stamp 
FROM '{topic_name}'
"""
print(sql)


SELECT name, 
    time_stamp, 
    sistolic, 
    diastolic, 
    id, 
    timestamp() as recv_stamp 
FROM 'notebook-topic'



The rule also requires an IAM Role authorizing it to write data on DynamoDB. The following trust relationship policy states that this role can be assumed by AWS IoT.

In [5]:
print(open('iot-trust.json').read())

{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "iot.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}


In [6]:
role_name = f"bp2dynamodb_role{HM}"
rule_role_arn = !aws iam create-role \
    --role-name {role_name} \
    --assume-role-policy-document file://iot-trust.json \
    --query Role.Arn \
    --output text
rule_role_arn = rule_role_arn.s
rule_role_arn

'arn:aws:iam::030555009967:role/bp2dynamodb_role0524'

And the rule policy states that it can insert data into this specific DynamoDB table

In [7]:
import json

In [8]:
rule_policy = {
    "Version": "2012-10-17",
    "Statement": {
        "Effect": "Allow",
        "Action": "dynamodb:PutItem",
        "Resource": table_arn
    }
}
rule_policy = json.dumps(rule_policy)
rule_policy

'{"Version": "2012-10-17", "Statement": {"Effect": "Allow", "Action": "dynamodb:PutItem", "Resource": "arn:aws:dynamodb:us-east-1:030555009967:table/bloodpressure0524"}}'

In [9]:
policy_name=f"bp2dynamodb_rule_policy{HM}"
policy_arn = !aws iam create-policy \
  --policy-name '{policy_name}' \
  --policy-document '{rule_policy}' \
  --query Policy.Arn \
  --output text
policy_arn = policy_arn.s
policy_arn

'arn:aws:iam::030555009967:policy/bp2dynamodb_rule_policy0524'

In [10]:
!aws iam attach-role-policy \
    --role-name {role_name} \
    --policy-arn {policy_arn}

In [11]:
rule = {
    "ruleDisabled": False,
    "sql": sql,
    "description": "Send blood preasure measurements to DynamoDB",
    "actions": [{
        "dynamoDBv2": {
            "roleArn": rule_role_arn,
            "putItem": {
                "tableName": table_name
            }
        }
    }]
}
rule = json.dumps(rule)
rule_file=f"{home}/rule.json" 
with open(rule_file, "w") as f:
    f.write(rule)
rule_file

'/Users/faermanj/rule.json'

In [12]:
print(open(rule_file).read())

{"ruleDisabled": false, "sql": "\nSELECT name, \n    time_stamp, \n    sistolic, \n    diastolic, \n    id, \n    timestamp() as recv_stamp \nFROM 'notebook-topic'\n", "description": "Send blood preasure measurements to DynamoDB", "actions": [{"dynamoDBv2": {"roleArn": "arn:aws:iam::030555009967:role/bp2dynamodb_role0524", "putItem": {"tableName": "bloodpressure0524"}}}]}


In [13]:
rule_name=f"bprule{HM}" 
!aws iot create-topic-rule \
  --rule-name '{rule_name}' \
  --topic-rule-payload file://{rule_file}

In [18]:
from AWSIoTPythonSDK.MQTTLib import AWSIoTMQTTClient

client_id=f"NotebookCient{HM}"
mqtt = AWSIoTMQTTClient(client_id)
mqtt.configureEndpoint(endpoint_address, 8883)
mqtt.configureCredentials(ca_file, private_key, certificate_file)
mqtt.configureConnectDisconnectTimeout(10)
mqtt.connect()
mqtt

In [21]:
import random
payload = random.choice(payloads)
print(payload)
mqtt.publish(topic_name, payload , QoS = 0)

{"id": "59f08348-4bf9-488b-9451-947e5c4b0d71", "name": "Jack Harkness", "sistolic": 139, "diastolic": 99, "time_stamp": 1517405078085}


True

Lets check how many records have arrived on DynamoDB...

In [22]:
!aws dynamodb scan --table-name {table_name} --query Count

3
[Errno 54] Connection reset by peer
[Errno 54] Connection reset by peer


In [17]:
mqtt.disconnect()

True

All right, but what if someone has high blood pressure, better let them know, right?
Let's do that in the [Reacting to Events notebook](aws-iot-react-events.ipynb)